<a href="https://colab.research.google.com/github/BENMEZIAN/Polars-tutorial/blob/main/Polars_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install polars using pip

In [1]:
!pip install polars

## Import polars

In [8]:
import polars as pl

data = {"column1": [1, 2, 3], "column2": ["a", "b", "c"]}
df = pl.DataFrame(data)

df

filtered_df = df.filter(pl.col("column1") > 1).sort("column2")

filtered_df

column1,column2
i64,str
2,"""b"""
3,"""c"""


## Real dataset

In [11]:
import polars as pl
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True).frame
df = pl.from_pandas(iris)

## General information

In [ ]:
# Some basic operations
print(df.head())

print("Dataset's dimensions :",df.shape)

print("Dataset's columns :",df.columns)

print("Dataset's columns types :",df.dtypes)

print("Dataset's rows count :",df.height) # print("Dataset's rows count :",len(df))

print("Dataset's general information :",df.describe())

#print("Dataset's memory usage :",df.memory_usage(deep=True))

## Select, filter, sort, join, ...

In [ ]:
# Select (simple syntax)
df.select(
    pl.col("sepal length (cm)"),
    pl.col("sepal width (cm)"),
    pl.col("petal length (cm)"),
    pl.col("petal width (cm)"),
    pl.col("target")
).head()

In [ ]:
# Rename
df.select(
    pl.col("sepal length (cm)").alias("sepal_length"),
    pl.col("sepal width (cm)").alias("sepal_width"),
    pl.col("petal length (cm)").alias("petal_length"),
    pl.col("petal width (cm)").alias("petal_width"),
    pl.col("target")
).head()

In [ ]:
# Add readable species labels (create/transform columns)
df = df.with_columns(
    pl.col("target")
      .cast(pl.Utf8)        # 👈 tell Polars strings are coming
      .replace({
          "0": "setosa",
          "1": "versicolor",
          "2": "virginica",
      })
      .alias("species")
)

print(df.head())

print(df.select("target", "species").unique())


In [ ]:
# Some simple filter()
records1 = df.filter(
    pl.col("sepal length (cm)") > 7
)

print(records1)

records2 = df.filter(
    (pl.col("sepal length (cm)") > 7) &
    (pl.col("sepal width (cm)") < 3)
)

print(records2)

records3 = df.filter(
    pl.col("species").is_in(["setosa", "versicolor"])
)

# I want to integrate select() with filter()
flowers = df.filter(
    pl.col("species").is_in(["setosa", "versicolor"])
).select(
    pl.col("sepal length (cm)"),
    pl.col("sepal width (cm)"),
    pl.col("species")
)

print(flowers)

starts_with_v = df.filter(pl.col("species").str.starts_with("v"))

print(starts_with_v)

not_null = df.filter(pl.col("sepal length (cm)").is_not_null())

print(not_null)

# Filter for rows where any numeric column has a value greater than 0
all_function = df.filter(pl.any_horizontal(pl.selectors.numeric() > 0))

all_function

## Aggregation functions

In [134]:
# Basic aggregation: mean of all numeric columns
df.select(pl.all().exclude("species")).mean()

# Aggregation grouped by species
df.group_by("species").mean()
df.group_by("species").max()
df.group_by("species").min()
df.group_by("species").median()
df.group_by("species").count()
df.group_by("species").sum()


# Multiple aggragation of calculated values
df.group_by("species").agg([
    pl.mean("sepal length (cm)").alias("avg_sepal_length"),
    pl.max("petal width (cm)").alias("max_petal_width"),
    pl.min("sepal width (cm)").alias("min_sepal_width")
])

# Lazy evaluation

result = (
    df.lazy()
    .group_by("species")
    .agg([
        pl.mean("sepal length (cm)").alias("avg_sepal_length"),
        pl.max("petal width (cm)").alias("max_petal_width")
    ])
    .collect()
)

print(result)

shape: (3, 3)
┌────────────┬──────────────────┬─────────────────┐
│ species    ┆ avg_sepal_length ┆ max_petal_width │
│ ---        ┆ ---              ┆ ---             │
│ str        ┆ f64              ┆ f64             │
╞════════════╪══════════════════╪═════════════════╡
│ virginica  ┆ 6.588            ┆ 2.5             │
│ setosa     ┆ 5.006            ┆ 0.6             │
│ versicolor ┆ 5.936            ┆ 1.8             │
└────────────┴──────────────────┴─────────────────┘


/tmp/ipython-input-3502348752.py:9: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  df.group_by("species").count()


## Example

In [133]:
# Give me the mean petal length only for species where sepal width > 3.0
print("First approach (simple)")
rows = df.filter(pl.col("sepal width (cm)") > 3.0)

result = rows.group_by("species").agg(
    pl.mean("petal length (cm)").alias("avg_petal_length")
).sort("avg_petal_length", descending=True) # ascending

print(result)

print("Second approach (lazy evaluation)")
lazy_approach = (
    df
    .filter(pl.col("sepal width (cm)") > 3.0)
    .group_by("species")
    .agg(pl.mean("petal length (cm)").alias("avg_petal_length"))
    .sort("avg_petal_length",descending = True) # ascending
)

print(lazy_approach)

First approach (simple)
shape: (3, 2)
┌────────────┬──────────────────┐
│ species    ┆ avg_petal_length │
│ ---        ┆ ---              │
│ str        ┆ f64              │
╞════════════╪══════════════════╡
│ virginica  ┆ 5.717647         │
│ versicolor ┆ 4.65             │
│ setosa     ┆ 1.480952         │
└────────────┴──────────────────┘
Second approach (lazy evaluation)
shape: (3, 2)
┌────────────┬──────────────────┐
│ species    ┆ avg_petal_length │
│ ---        ┆ ---              │
│ str        ┆ f64              │
╞════════════╪══════════════════╡
│ virginica  ┆ 5.717647         │
│ versicolor ┆ 4.65             │
│ setosa     ┆ 1.480952         │
└────────────┴──────────────────┘


In [132]:
# Among flowers with sepal_width > 3.0, show species with the highest average petal length, along with the max petal width
top_species = (
    df
    .filter(pl.col("sepal width (cm)") > 3.0)
    .group_by("species")
    .agg([
        pl.mean("petal length (cm)").alias("avg_petal_length"),
        pl.max("petal width (cm)").alias("max_petal_width")
    ])
    .sort("avg_petal_length", descending=True)
    .head(1)
)

print(top_species)

shape: (1, 3)
┌───────────┬──────────────────┬─────────────────┐
│ species   ┆ avg_petal_length ┆ max_petal_width │
│ ---       ┆ ---              ┆ ---             │
│ str       ┆ f64              ┆ f64             │
╞═══════════╪══════════════════╪═════════════════╡
│ virginica ┆ 5.717647         ┆ 2.5             │
└───────────┴──────────────────┴─────────────────┘


## Save I/O

In [137]:
df = pl.read_csv('/content/sample_data/mnist_test.csv')

# Display the first few rows
print(df.head())


##########################################
# pl.read_csv
# pl.write.csv
# pl.scan_csv("iris_polars.csv")  # lazy read (recommended for efficiency)

# pl.read_parquet (recommended)
# pl.write_parquest
##########################################

shape: (5, 785)
┌─────┬─────┬─────────────┬─────────────┬───┬─────────────┬─────────────┬─────────────┬────────────┐
│ 7   ┆ 0   ┆ 0_duplicate ┆ 0_duplicate ┆ … ┆ 0_duplicate ┆ 0_duplicate ┆ 0_duplicate ┆ 0_duplicat │
│ --- ┆ --- ┆ d_0         ┆ d_1         ┆   ┆ d_663       ┆ d_664       ┆ d_665       ┆ ed_666     │
│ i64 ┆ i64 ┆ ---         ┆ ---         ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---        │
│     ┆     ┆ i64         ┆ i64         ┆   ┆ i64         ┆ i64         ┆ i64         ┆ i64        │
╞═════╪═════╪═════════════╪═════════════╪═══╪═════════════╪═════════════╪═════════════╪════════════╡
│ 2   ┆ 0   ┆ 0           ┆ 0           ┆ … ┆ 0           ┆ 0           ┆ 0           ┆ 0          │
│ 1   ┆ 0   ┆ 0           ┆ 0           ┆ … ┆ 0           ┆ 0           ┆ 0           ┆ 0          │
│ 0   ┆ 0   ┆ 0           ┆ 0           ┆ … ┆ 0           ┆ 0           ┆ 0           ┆ 0          │
│ 4   ┆ 0   ┆ 0           ┆ 0           ┆ … ┆ 0           ┆ 0           ┆ 0